In [ ]:
import pandas as pd, boto3, json, re
from io import BytesIO
from datetime import datetime

read csv from s3 object


In [ ]:
with open("/home/jh97/MyWorks/Documents/.aws_cdt.json", "r") as file:
    content = json.load(file)
    key_id = content["access_key"]
    secret_key = content["secrect_access_key"]

client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
)

# panigator = client.get_paginator("list_objects_v2")
# result = panigator.paginate(Bucket="crawling-to-datalake", Prefix="scraped/")

# for i in result.build_full_result()["Contents"]:
#     if datetime.fromisoformat(
#         i["Key"].split("_")[-1].split(".")[0]
#     ) < datetime.now() and i["Key"].split("_")[0].endswith("tgdd"):
#         print(i["Key"])
#         break

resp = client.get_object(
    Bucket="crawling-to-datalake", Key="scraped/tgdd_products_2025-08-26.csv"
)

df = pd.read_csv(BytesIO(resp["Body"].read()))
df

concat 2 dataframes


In [ ]:
df = pd.concat(
    [
        pd.read_csv("../data/scraped/tgdd_products_2025-08-29.csv"),
        pd.read_csv("../data/scraped/cellphones_products_2025-08-29.csv"),
    ]
)
df

cleaning


In [ ]:
# change dtypes
df = df.astype({"onsale_price": "Int64", "reviews_count": "Int64"})
df.dtypes

In [ ]:
def normalize_name(name: str):
    name = name.upper().replace(".", "-").replace("/", "-").replace(" INCH", '"')
    
    noises = [
        r"^(LAPTOP|ĐIỆN THOẠI (TRẺ EM)?)",
        r"\s+\(.*\).*",
        r"\s+\d+(MB|GB|TB).*",
        r"(COPILOT|CHÍNH HÃNG|NHẬP KHẨU).*",
        r"\d+\s?CPU.*",
        r"\s+IR.*",
        r"\s?-.*$",
        # r'\s\b[A-Z0-9]{4,10}\b$'
    ]

    for i in noises:
        name = re.sub(i, "", name)

    name = re.sub(r"\bGEN (\d\d?)\b", r"G\1", name)
    name = re.sub(r"\S(PRO|FE)", r" \1", name)

    return name.strip()


df["name"].map(normalize_name).sort_values().unique()

array(['ACER ASPIRE 16 AI A16 71M 59L5', 'ACER ASPIRE 16 AI A16 71M 71U7',
       'ACER ASPIRE 3 A314', ..., 'ZTE BLADE V50 DESIGN',
       'ZTE NUBIA Z60 ULTRA', 'ZTE NUBIA Z60S PRO'],
      shape=(1260,), dtype=object)

In [ ]:
# df["brand"].where(
#     ~df["brand"].str.contains(r"iphone|macbook", regex=True, case=False), other="Apple"
# )

# df["os"].str.replace(r"\s+\+.*|\s+\(.*", "", regex=True, case=False)

# df["cpu"].str.replace(r"-\sHãng.*|chuỗi N\s+", "", regex=True, case=False)


# df["gpu"][
#     ~df["gpu"].astype(str).str.contains(r"rời|NVIDIA", regex=True, case=False)
# ].unique()

# df["gpu"].map(
#     lambda x: (
#         (
#             x + ", Discrete"
#             if ("NVIDIA" in str(x) or "rời" in str(x))
#             else x + ", Integrated"
#         )
#         if isinstance(x, str)
#         else None
#     )
# ).str.replace(r".*\s+-\s+|\s+eligible|\s+\(.*\)", "", regex=True, case=False).unique()